# Introduction to SALib: Ishigami function

Interactive examples for the clinic, Model Sensitivity Analysis with SALib at [CSDMS Annual Meeting 2019](https://csdms.colorado.edu/wiki/Form:Annualmeeting2019).

**Goal of this notebook:** Use the Ishigami function to demonstrate the Sobol method implemented in SALib.

Clinic resources are stored in [this repository](https://github.com/nathanlyons/sensitivity_analysis_clinic_CSDMS_2019).

### How to use this notebook

This notebook is made of:
* text cells, including the cell you are reading now, and
* code cells where computations are run and figures are created.

The order in which you run the cells matters. The cells of this notebook are designed to be run top to bottom unless stated otherwise. 

**To run a cell:**
1. Click a cell with your cursor.
2. Hold down `shift` and then also press `enter`.

Code cells have the following characters to their left:
* `In [ ]:` is a cell yet to be run.
* `In [*]:` is a cell that is running.
* `In [n]:` where `n` is an integer, is a cell that has run. The number indicates the sequence the cell was run, i.e. the first cell run is `n`=1.

## 1. Prepare the environment
Run the code cell below by selecting it, then hold down `shift` and then also press `enter`.

In [ ]:
import math
from pprint import pprint

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from SALib.analyze import sobol
from SALib.sample import saltelli

import plot_funcs as pf

Run the next 2 cells as well to make it easier to see and work with plots later in this notebook.

In [ ]:
%matplotlib notebook

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## 2. Ishigami function


The Ishigami function (ref 1):

\begin{equation*}
f(\mathbf{x})   = \sin(x_1) + a \sin^2(x_2) + bx_3^4 \sin(x_1)
\end{equation*}

* Often used example for uncertainty and sensitivity analysis methods.
* Exhibits strong nonlinearity and nonmonotonicity.
* Ref 2 uses $a$ = 7 and $b$ = 0.05.

Refs
1. Ishigami T, Homma  T, 1990. An importance quantification technique in uncertainty analysis for computer models. First International Symposium on Uncertainty Modeling and Analysis, Proceedings. pp. 398-403. IEEE.
2. Sobol IM, Levitan YL, 1999. On the use of variance reducing multipliers in Monte Carlo computations of a global sensitivity index. Computer Physics Communications, 117, 52-61.

## 3. Prepare the analysis

SALib requires a `problem` dictionary with the key-value pairs below.

Here we will create a `problem` dictionary to perform a sensitivity analysis of the Ishigami function with each of the three factor values spanning from -$\pi$ to $\pi$.

In [ ]:
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': 3 * [[-math.pi, math.pi]]
}

pprint(problem)

### Generate model inputs

The number of model inputs (i.e. factor sets), is equal to $N(2D + 2)$, where $N$ is the number of samples and $D$ is the number of parameters.

In a Jupyter Notebook you can view method documention by:
* entering a `?` after method, or
* place your cursor in the method name, and then press `shift`+`tab`+`tab` to get the documentation.

In [ ]:
saltelli.sample?

Generate model inputs

In [ ]:
number_of_samples = 1000

factor_sets = saltelli.sample(problem, number_of_samples)

print('number of factor sets:', len(factor_sets))
print('N(2D + 2):', number_of_samples * (2 * problem['num_vars'] + 2))

### Plot parameter space

In [ ]:
xt = np.array(factor_sets).transpose()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xt[0], xt[1], xt[2], s=10)

ticks = [-np.pi, 0, np.pi]
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_zticks(ticks)

labels = ['-$\pi$', 0, '$\pi$']
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)
ax.set_zticklabels(labels)

ax.set_xlabel('$x_1$', color='r', labelpad=10)
ax.set_ylabel('$x_2$', color='r', labelpad=10)
ax.set_zlabel('$x_3$', color='r', labelpad=10)

## 4. Perform sensitivity analysis

We will explore the Sobol method by running the Ishigami function/model. We will iteratively include more terms of the function as we go. (Use the same `problem` dictionary above.)

**1 term**: $f(\mathbf{x}) = \sin(x_1)$

**2 terms**: $f(\mathbf{x}) = \sin(x_1) + a \sin^2(x_2)$

**3 terms**: $f(\mathbf{x}) = \sin(x_1) + a \sin^2(x_2) + bx_3^4 \sin(x_1)$

In [ ]:
term_count = 1

In [ ]:
# Create an array to store the response of each trial.
n_trials = factor_sets.shape[0]
Y = np.zeros(n_trials)

# Set constants.
A = 7
B = 0.05

# Calculate the output, `Y` for each factor set.
for i, X in enumerate(factor_sets):
    
    if term_count == 1:
        Y[i] = math.sin(X[0])
        
    elif term_count == 2:
        Y[i] = math.sin(X[0]) + A * math.pow(math.sin(X[1]), 2)
        
    elif term_count == 3:
        Y[i] = math.sin(X[0]) + A * math.pow(math.sin(X[1]), 2) + B * math.pow(X[2], 4) * math.sin(X[0])
        
    else:
        print('You entered an invalid value for `term_count`.')
        
Si = sobol.analyze(problem, Y)

pf.plot_sobol_indices(Si, problem)

Print results.

In [ ]:
pprint(Si.to_df())

**Return to the beginning of Section 4 and try different term counts.** 